# AnacondaでMayaviを使う方法 / How to use Mayavi in Anaconda



1. Anaconda Navigatorで左メニューのEnvironmentsを押し、Create+を押して、Mayaviという名前の新しい環境を作る。環境とは、Pythonライブラリ(パッケージ)を集めたもの。標準はroot環境だが、mayaviは最新環境(3.6)では動かないので、Python3.5を選択する。
2. 最初は最小限(10個)のパッケージしか含まれていない。
2. Mayaviの緑の三角">"を押してOpen Terminalするとターミナルが開く。
3. その中で、OpenCVに必要なものをインストールする。
       conda install -c menpo mayavi=4.5.0
       conda install -c conda-forge pyside=1.2.4
   これにより、新しい環境(Python3.5, Mayavi入り)の準備ができた。今後、また別の開発環境が欲しくなった時も同じ手順を踏む。
5. Anaconda Navigatorに戻り、左メニュー「Home」でアプリ起動画面を表示する。「Applications on」でrootの代わりにMayaviを選ぶと、使えるアプリケーションが更新される。これらのアプリケーションすべてで、Mayaviが利用できる。


1. Press "Environments" in the left menu of the Anaconda Navigator, press "Create[+]" button at the bottom, and make a new environment of Python version 3.5 with a new name "mayavi".  An environment is a collection of packages (libraries).  The standard environment is named "root" and is working on Python version 3.6, but Mayavi library work on Python version 3.5 only, so we prepare another environment.
2. Initially, only 10 packages are included in the Mayavi environment.
3. Press the green triangle at "Mayavi" and select "Open Terminal".
4. In the terminal, type the following command to install the packages required to Mayavi.
        conda install -c menpo mayavi=4.5.0
        conda install -c conda-forge pyside=1.2.4
5. Back to Anaconda Navigator, press "Home" in the left menu to go back the home screen, and select "mayavi" in the "Applications on" selector.  It refreshes the menu item of the Navigator.  You can now use Mayavi in Jupyter and spyder.

### Notes

* https://anaconda.org/menpo/mayavi
* https://github.com/enthought/mayavi/issues/483

In [1]:
import numpy as np
from mayavi.mlab import *
import colorsys



def tangents(x,y,z):
    v = np.array([x,y,z]).transpose()
    v0 = v[0:v.shape[0]-2]
    v1 = v[2:]
    dv = v1 - v0
    tangent = np.zeros_like(v)
    for i in range(dv.shape[0]):
        tangent[i+1] = dv[i] / np.linalg.norm(dv[i])
    tangent[0] = tangent[1]
    tangent[-1] = tangent[-2]
    return tangent

def normals(x,y,z):
    v = np.array([x,y,z]).transpose()
    v0 = v[0:v.shape[0]-1]
    v1 = v[1:]
    dv = v1 - v0
    dv0 = dv[0:dv.shape[0]-1]
    dv1 = dv[1:]
    n0 = np.cross(dv0,dv1)
    print(v0[0:3])
    print(v1[0:3])
    print(dv[0:3])
    print(n0[0:3])
    n1 = np.zeros_like(v)
    for i in range(n0.shape[0]):
        n0size = np.linalg.norm(n0[i])
        if n0size < 1e-9:
            n1[i+1] = n1[i]
        else:
            n1[i+1] = n0[i] / np.linalg.norm(n0[i])
    n1[0] = n1[1]
    n1[-1] = n1[-2]
    return n1


def ribbon(x,y,z,width,twist, arrow_rows=0, arrow_expansion=1.0):
    norm = normals(x,y,z)
    tang = tangents(x,y,z)
    #print(norm)
    v3   = np.cross(norm,tang)
    #plot3d(x,y,z)
    v = np.array([x,y,z]).transpose()
    sine   = np.sin(twist)
    cosine = np.cos(twist)
    direction = np.zeros_like(v)
    for i in range(v.shape[0]):
        direction[i] = width[i]*(cosine[i]*norm[i,:] + sine[i]*v3[i,:])
    ps = []
    qs = []
    for i in range(v.shape[0]):
        j = v.shape[0] - i - 1
        if j <= arrow_rows and arrow_rows != 0:
            arrow = arrow_expansion*j/arrow_rows
        else:
            arrow = 1
        p = v[i]-direction[i]*arrow/2
        q = v[i]+direction[i]*arrow/2
        ps.append(p)
        qs.append(q)
    qs.reverse()
    pq = ps + qs
    
    triplets = []
    for p in range(v.shape[0]-1):
        q = v.shape[0]*2 - p - 1
        triplets.append([p,p+1,q])
        triplets.append([p+1,q-1,q])
    xx,yy,zz = np.array(pq).transpose()
    return xx,yy,zz,triplets
        
        #xx,yy,zz = np.array([p,q]).transpose()
        #plot3d(xx,yy,zz)


def ribbon_arrow():
    t = np.arange(-1.0, 2.0, 0.02)
    t1 = 0<=t
    t2 = t<1
    curve = np.logical_and(t1,t2)
    pre   = t < 0
    post  = t >= 1
    twist = np.zeros_like(t)
    width = t*0+3
    x = np.zeros_like(t)
    y = np.zeros_like(t)
    z = np.zeros_like(t)
    r = 3.0
    pi = np.pi
    x[curve] = r*np.cos(t[curve]*pi/2)
    y[curve] = r*np.sin(t[curve]*pi/2)
    z[curve] = r*np.cos(t[curve]*pi)
    x[pre]   = r
    y[pre]   = t[pre]
    z[pre]   = r
    z[0]     = r+0.003
    x[post]  = -r*(t[post]-1)
    y[post]  = r
    z[post]  = -r
    plot3d(x,y,z)
    tx,ty,tz,ta = ribbon(x,y,z,width,twist,arrow_rows=60,arrow_expansion=2.0)
#    tx,ty,tz,ta = ribbon(x,y,z,width,twist)
    plot3d(tx,ty,tz)
    l = triangular_mesh(tx, ty, tz, ta)
    return l

clf()
#ribbon_arrow()
#spiral_tower(1,1,4)
points3d(0,0,60,1,color=(1,1,1))
#test_points3d()

In [3]:
import pyside

ImportError: No module named 'pyside'

In [4]:
import numpy
from mayavi.mlab import *

def test_barchart():
    """ Demo the bar chart plot with a 2D array.
    """
    s = numpy.abs(numpy.random.random((3, 3)))
    return barchart(s)

test_barchart()